# Проект: **Музыка больших городов**

## Получение данных



In [1]:
import pandas as pd # импорт библиотеки pandas

Прочитаем файл yandex_music_project.csv из папки /datasets и сохраните его в переменной df:

In [2]:
df = pd.read_csv('https://code.s3.yandex.net/datasets/yandex_music_project.csv') # чтение файла с данными и сохранение в df

Получение первых 10 строк таблицы.

In [3]:
df.head(10)# получение первых 10 строк таблицы df

,userID,Track,artist,genre,City,time,Day
0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Saint-Petersburg,20:28:33,Wednesday
1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Moscow,14:07:09,Friday
2,20EC38,Funiculì funiculà,Mario Lanza,pop,Saint-Petersburg,20:58:07,Wednesday
3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Saint-Petersburg,08:37:09,Monday
4,E2DC1FAE,Soul People,Space Echo,dance,Moscow,08:34:34,Monday
5,842029A1,Преданная,IMPERVTOR,rusrap,Saint-Petersburg,13:09:41,Friday
6,4CB90AA5,True,Roman Messer,dance,Moscow,13:00:07,Wednesday
7,F03E1C1F,Feeling This Way,Polina Griffith,dance,Moscow,20:47:49,Wednesday
8,8FA1D3BE,И вновь продолжается бой,NaN,ruspop,Moscow,09:17:40,Friday
9,E772D5C0,Pessimist,NaN,dance,Saint-Petersburg,21:20:49,Wednesday


Получим общую информацию о таблице c помощью метода info():

In [4]:
df.info()# получение общей информации о данных в таблице df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65079 entries, 0 to 65078
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0     userID  65079 non-null  object
 1   Track     63848 non-null  object
 2   artist    57876 non-null  object
 3   genre     63881 non-null  object
 4     City    65079 non-null  object
 5   time      65079 non-null  object
 6   Day       65079 non-null  object
dtypes: object(7)
memory usage: 3.5+ MB


Итак, в таблице семь столбцов. Тип данных во всех столбцах — `object`.

Согласно документации к данным:
* `userID` — идентификатор пользователя;
* `Track` — название трека;  
* `artist` — имя исполнителя;
* `genre` — название жанра;
* `City` — город пользователя;
* `time` — время начала прослушивания;
* `Day` — день недели.

Количество значений в столбцах различается. Значит, в данных есть пропущенные значения.

*Промежуточные выводы*<br>
Каждая строка таблицы содержит информацию о композициях определённого жанра в определённом исполнении, которые пользователи слушали в одном из городов в определённое время и день недели. Две проблемы, которые нужно решать: пропуски и некачественные названия столбцов. Для проверки рабочих гипотез особенно ценны столбцы time, day и City. Данные из столбца genre позволят узнать самые популярные жанры.<br>
## Предобработка данных

Исключим пропуски, переименуем столбцы, а также проверим данные на наличие дубликатов.

In [5]:
df = df.rename(columns={'  userID':'user_id','Track':'track','  City  ':'city','Day':'day'})# переименование столбцов

Проверим данные на наличие пропусков вызовом набора методов для суммирования пропущенных значений.

In [6]:
df.isna().sum()# подсчёт пропусков

user_id       0
track      1231
artist     7203
genre      1198
city          0
time          0
day           0
dtype: int64

Пустые значения свидетельствуют, что для некоторых треков доступна не вся информация. Причины могут быть разные: скажем, не назван конкретный исполнитель народной песни. Хуже, если проблемы с записью данных. Каждый отдельный случай необходимо разобрать и выявить причину.

Не все пропущенные значения влияют на исследование. Так в `track` и `artist` пропуски не важны для вашей работы. Достаточно заменить их явными обозначениями.

Но пропуски в `genre` могут помешать сравнению музыкальных вкусов в Москве и Санкт-Петербурге. На практике было бы правильно установить причину пропусков и восстановить данные. Такой возможности нет в учебном проекте. Придётся:
* заполнить и эти пропуски явными обозначениями,
* оценить, насколько они повредят расчётам. 

Заменяем пропущенные значения в столбцах с названием трека, исполнителя и жанра на строку 'unknown'.

In [7]:
# перебор названий столбцов в цикле и замена пропущенных значений на 'unknown'
columns_to_replace = ['track','artist','genre']
for column in columns_to_replace:
    df[column] = df[column].fillna('unknown')

Убедимся, что мы избались от всех пропусков

In [8]:
df.isnull().sum()# <проверка>

user_id    0
track      0
artist     0
genre      0
city       0
time       0
day        0
dtype: int64

Необходимо установить наличие явных дубликатов.

In [9]:
df.duplicated().sum()# подсчёт явных дубликатов

3826

In [10]:
df = df.drop_duplicates()# удаление явных дубликатов (с удалением старых индексов и формированием новых)

Ещё раз посчитаем явные дубликаты в таблице — убедимся, что полностью от них избавились:

In [11]:
df.duplicated().sum()# проверка на отсутствие дубликатов

0

Теперь избавимся от неявных дубликатов в колонке genre.

In [12]:
# Просмотр уникальных названий жанров
df['genre'].sort_values().unique()

array(['acid', 'acoustic', 'action', 'adult', 'africa', 'afrikaans',
       'alternative', 'alternativepunk', 'ambient', 'americana',
       'animated', 'anime', 'arabesk', 'arabic', 'arena',
       'argentinetango', 'art', 'audiobook', 'author', 'avantgarde',
       'axé', 'baile', 'balkan', 'beats', 'bigroom', 'black', 'bluegrass',
       'blues', 'bollywood', 'bossa', 'brazilian', 'breakbeat', 'breaks',
       'broadway', 'cantautori', 'cantopop', 'canzone', 'caribbean',
       'caucasian', 'celtic', 'chamber', 'chanson', 'children', 'chill',
       'chinese', 'choral', 'christian', 'christmas', 'classical',
       'classicmetal', 'club', 'colombian', 'comedy', 'conjazz',
       'contemporary', 'country', 'cuban', 'dance', 'dancehall',
       'dancepop', 'dark', 'death', 'deep', 'deutschrock', 'deutschspr',
       'dirty', 'disco', 'dnb', 'documentary', 'downbeat', 'downtempo',
       'drum', 'dub', 'dubstep', 'eastern', 'easy', 'electronic',
       'electropop', 'emo', 'entehno', '

Выявлены неявные дубликаты названия `hiphop`:

* *hip*,
* *hop*,
* *hip-hop*.

Это могут быть названия с ошибками или альтернативные названия того же жанра.

Чтобы очистить от них таблицу используем метод `replace()`:

In [13]:
# Устранение неявных дубликатов
wrong_name = ['hip','hop','hip-hop']
df['genre'] = df['genre'].replace(wrong_name,'hiphop')

*Промежуточные выводы*


Предобработка обнаружила три проблемы в данных:

- нарушения в стиле заголовков,
- пропущенные значения,
- дубликаты — явные и неявные.

Мы исправили заголовки, чтобы упростить работу с таблицей. Без дубликатов исследование станет более точным.

Пропущенные значения мы заменили на `'unknown'`. Ещё предстоит увидеть, не повредят ли исследованию пропуски в колонке `genre`.

Теперь можно перейти к проверке гипотез. 
## Проверка гипотез
### Сравнение поведения пользователей двух столиц

Первая гипотеза утверждает, что пользователи по-разному слушают музыку в Москве и Санкт-Петербурге. Проверим это предположение по данным о трёх днях недели — понедельнике, среде и пятнице. Для этого:

* Разделим пользователей Москвы и Санкт-Петербурга
* Сравним, сколько треков послушала каждая группа пользователей в понедельник, среду и пятницу.

In [14]:
# Подсчёт прослушиваний в каждом городе
df.groupby('city')['track'].count()

city
Moscow              42741
Saint-Petersburg    18512
Name: track, dtype: int64

В Москве прослушиваний больше, чем в Петербурге. Из этого не следует, что московские пользователи чаще слушают музыку. Просто самих пользователей в Москве больше.

In [15]:
# Подсчёт прослушиваний в каждый из трёх дней
df.groupby('day')['track'].count()

day
Friday       21840
Monday       21354
Wednesday    18059
Name: track, dtype: int64

В среднем пользователи из двух городов менее активны по средам. Но картина может измениться, если рассмотреть каждый город в отдельности.

Создадим функцию `number_tracks()`, которая посчитает прослушивания для заданного дня и города. Ей понадобятся два параметра:
* день недели,
* название города.

В функции сохраним в переменную строки исходной таблицы, у которых значение:
  * в колонке `day` равно параметру `day`,
  * в колонке `city` равно параметру `city`.

Для этого применим последовательную фильтрацию с логической индексацией.

Затем посчитаем значения в столбце `user_id` получившейся таблицы. Результат сохраним в новую переменную.

In [16]:
def number_tracks(day, city):# <создание функции number_tracks()>
    track_list = df[df['day']==day]# Объявляется функция с двумя параметрами: day, city.
    track_list = track_list[df['city']==city]# В переменной track_list сохраняются те строки таблицы df, для которых 
    track_list_count = track_list['user_id'].count()
    return track_list_count

In [17]:
number_tracks('Monday', 'Moscow')# количество прослушиваний в Москве по понедельникам

/var/folders/jf/fc0h52cn4szf8thmzw2fn77r0000gn/T/ipykernel_92765/2982332760.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  track_list = track_list[df['city']==city]# В переменной track_list сохраняются те строки таблицы df, для которых


15740

In [18]:
number_tracks('Monday', 'Saint-Petersburg')# количество прослушиваний в Санкт-Петербурге по понедельникам

/var/folders/jf/fc0h52cn4szf8thmzw2fn77r0000gn/T/ipykernel_92765/2982332760.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  track_list = track_list[df['city']==city]# В переменной track_list сохраняются те строки таблицы df, для которых


5614

In [19]:
number_tracks('Wednesday', 'Moscow')# количество прослушиваний в Москве по средам

/var/folders/jf/fc0h52cn4szf8thmzw2fn77r0000gn/T/ipykernel_92765/2982332760.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  track_list = track_list[df['city']==city]# В переменной track_list сохраняются те строки таблицы df, для которых


11056

In [20]:
number_tracks('Wednesday', 'Saint-Petersburg')# количество прослушиваний в Санкт-Петербурге по средам

/var/folders/jf/fc0h52cn4szf8thmzw2fn77r0000gn/T/ipykernel_92765/2982332760.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  track_list = track_list[df['city']==city]# В переменной track_list сохраняются те строки таблицы df, для которых


7003

In [21]:
number_tracks('Friday', 'Moscow')# количество прослушиваний в Москве по пятницам

/var/folders/jf/fc0h52cn4szf8thmzw2fn77r0000gn/T/ipykernel_92765/2982332760.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  track_list = track_list[df['city']==city]# В переменной track_list сохраняются те строки таблицы df, для которых


15945

In [22]:
number_tracks('Friday', 'Saint-Petersburg')# количество прослушиваний в Санкт-Петербурге по пятницам

/var/folders/jf/fc0h52cn4szf8thmzw2fn77r0000gn/T/ipykernel_92765/2982332760.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  track_list = track_list[df['city']==city]# В переменной track_list сохраняются те строки таблицы df, для которых


5895


Создадим c помощью конструктора `pd.DataFrame` таблицу, где
* названия колонок — `['city', 'monday', 'wednesday', 'friday']`;
* данные — результаты, которые вы получили с помощью `number_tracks`.

In [23]:
columns = ['city', 'monday', 'wednesday', 'friday']# Таблица с результатами
data = [
    ['Moscow', 15740, 11056, 15945],
    ['Saint-Petersburg', 5614, 7003, 5895]
]
pd.DataFrame(data=data,columns=columns)

,city,monday,wednesday,friday
0,Moscow,15740,11056,15945
1,Saint-Petersburg,5614,7003,5895


*Промежуточные выводы*

Данные показывают разницу поведения пользователей:

- В Москве пик прослушиваний приходится на понедельник и пятницу, а в среду заметен спад.
- В Петербурге, наоборот, больше слушают музыку по средам. Активность в понедельник и пятницу здесь почти в равной мере уступает среде.

Значит, данные говорят в пользу первой гипотезы.

### Музыка в начале и в конце недели

Согласно второй гипотезе, утром в понедельник в Москве преобладают одни жанры, а в Петербурге — другие. Так же и вечером пятницы преобладают разные жанры — в зависимости от города.

In [24]:
# получение таблицы moscow_general из тех строк таблицы df, 
# для которых значение в столбце 'city' равно 'Moscow'
moscow_general = df[df['city']=='Moscow']

In [25]:
# получение таблицы spb_general из тех строк таблицы df,
# для которых значение в столбце 'city' равно 'Saint-Petersburg'
spb_general = df[df['city']=='Saint-Petersburg']

Создадим функцию `genre_weekday()` с четырьмя параметрами:
* таблица (датафрейм) с данными,
* день недели,
* начальная временная метка в формате 'hh:mm', 
* последняя временная метка в формате 'hh:mm'.

Функция должна вернуть информацию о топ-10 жанров тех треков, которые прослушивали в указанный день, в промежутке между двумя отметками времени.

In [26]:
def genre_weekday(df, day, time1, time2):
    # последовательная фильтрация
    # оставляем в genre_df только те строки df, у которых день равен day
    genre_df = df[df['day']==day]
    # оставляем в genre_df только те строки genre_df, у которых время меньше time2
    genre_df = genre_df[df['time'] < time2]
    # оставляем в genre_df только те строки genre_df, у которых время больше time1
    genre_df = genre_df[df['time'] > time1]
    # сгруппируем отфильтрованный датафрейм по столбцу с названиями жанров, возьмём столбец genre и посчитаем кол-во строк для каждого жанра методом count()
    genre_df_grouped = genre_df.groupby('genre')['genre'].count()
    # отсортируем результат по убыванию (чтобы в начале Series оказались самые популярные жанры)
    genre_df_sorted = genre_df_grouped.sort_values(ascending = False)
    # вернём Series с 10 самыми популярными жанрами в указанный отрезок времени заданного дня
    return genre_df_sorted[:10]

Cравним результаты функции `genre_weekday()` для Москвы и Санкт-Петербурга в понедельник утром (с 7:00 до 11:00) и в пятницу вечером (с 17:00 до 23:00):

In [27]:
# вызов функции для утра понедельника в Москве 
genre_weekday(moscow_general, 'Monday', '07:00', '11:00')

/var/folders/jf/fc0h52cn4szf8thmzw2fn77r0000gn/T/ipykernel_92765/1354905994.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  genre_df = genre_df[df['time'] < time2]
/var/folders/jf/fc0h52cn4szf8thmzw2fn77r0000gn/T/ipykernel_92765/1354905994.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  genre_df = genre_df[df['time'] > time1]


genre
pop            781
dance          549
electronic     480
rock           474
hiphop         286
ruspop         186
world          181
rusrap         175
alternative    164
unknown        161
Name: genre, dtype: int64

In [28]:
# вызов функции для утра понедельника в Петербурге
genre_weekday(spb_general, 'Monday', '07:00', '11:00')

/var/folders/jf/fc0h52cn4szf8thmzw2fn77r0000gn/T/ipykernel_92765/1354905994.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  genre_df = genre_df[df['time'] < time2]
/var/folders/jf/fc0h52cn4szf8thmzw2fn77r0000gn/T/ipykernel_92765/1354905994.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  genre_df = genre_df[df['time'] > time1]


genre
pop            218
dance          182
rock           162
electronic     147
hiphop          80
ruspop          64
alternative     58
rusrap          55
jazz            44
classical       40
Name: genre, dtype: int64

In [29]:
# вызов функции для вечера пятницы в Москве
genre_weekday(moscow_general, 'Friday', '17:00', '23:00')

/var/folders/jf/fc0h52cn4szf8thmzw2fn77r0000gn/T/ipykernel_92765/1354905994.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  genre_df = genre_df[df['time'] < time2]
/var/folders/jf/fc0h52cn4szf8thmzw2fn77r0000gn/T/ipykernel_92765/1354905994.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  genre_df = genre_df[df['time'] > time1]


genre
pop            713
rock           517
dance          495
electronic     482
hiphop         273
world          208
ruspop         170
alternative    163
classical      163
rusrap         142
Name: genre, dtype: int64

In [30]:
# вызов функции для вечера пятницы в Петербурге
genre_weekday(spb_general, 'Friday', '17:00', '23:00')

/var/folders/jf/fc0h52cn4szf8thmzw2fn77r0000gn/T/ipykernel_92765/1354905994.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  genre_df = genre_df[df['time'] < time2]
/var/folders/jf/fc0h52cn4szf8thmzw2fn77r0000gn/T/ipykernel_92765/1354905994.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  genre_df = genre_df[df['time'] > time1]


genre
pop            256
electronic     216
rock           216
dance          210
hiphop          97
alternative     63
jazz            61
classical       60
rusrap          59
world           54
Name: genre, dtype: int64

*Промежуточные выводы*

Если сравнить топ-10 жанров в понедельник утром, можно сделать такие выводы:

1. В Москве и Петербурге слушают похожую музыку. Единственное отличие — в московский рейтинг вошёл жанр “world”, а в петербургский — джаз и классика.

2. В Москве пропущенных значений оказалось так много, что значение `'unknown'` заняло десятое место среди самых популярных жанров. Значит, пропущенные значения занимают существенную долю в данных и угрожают достоверности исследования.

Вечер пятницы не меняет эту картину. Некоторые жанры поднимаются немного выше, другие спускаются, но в целом топ-10 остаётся тем же самым.

Таким образом, вторая гипотеза подтвердилась лишь частично:
* Пользователи слушают похожую музыку в начале недели и в конце.
* Разница между Москвой и Петербургом не слишком выражена. В Москве чаще слушают русскую популярную музыку, в Петербурге — джаз.

Однако пропуски в данных ставят под сомнение этот результат. В Москве их так много, что рейтинг топ-10 мог бы выглядеть иначе, если бы не утерянные  данные о жанрах.

### Жанровые предпочтения в Москве и Петербурге

Гипотеза: Петербург — столица рэпа, музыку этого жанра там слушают чаще, чем в Москве.  А Москва — город контрастов, в котором, тем не менее, преобладает поп-музыка.

In [31]:
moscow_genres = moscow_general.groupby('genre')['genre'].count().sort_values( ascending = False)

In [32]:
moscow_genres.head(10) # Топ-10 жанров в Москве

genre
pop            5892
dance          4435
rock           3965
electronic     3786
hiphop         2096
classical      1616
world          1432
alternative    1379
ruspop         1372
rusrap         1161
Name: genre, dtype: int64

In [33]:
spb_genres = spb_general.groupby('genre')['genre'].count().sort_values( ascending = False)

In [34]:
spb_genres.head(10)# Топ-10 жанров в СПБ

genre
pop            2431
dance          1932
rock           1879
electronic     1736
hiphop          960
alternative     649
classical       646
rusrap          564
ruspop          538
world           515
Name: genre, dtype: int64

*Промежуточные выводы*
Гипотеза частично подтвердилась:
* Поп-музыка — самый популярный жанр в Москве, как и предполагала гипотеза. Более того, в топ-10 жанров встречается близкий жанр — русская популярная музыка.
* Вопреки ожиданиям, рэп одинаково популярен в Москве и Петербурге. 

## Итоги исследования 

В результате проверки трех гипотез, установлено:

1. День недели по-разному влияет на активность пользователей в Москве и Петербурге. 

Первая гипотеза полностью подтвердилась.

2. Музыкальные предпочтения не сильно меняются в течение недели — будь то Москва или Петербург. Небольшие различия заметны в начале недели, по понедельникам:
* в Москве слушают музыку жанра “world”,
* в Петербурге — джаз и классику.

Таким образом, вторая гипотеза подтвердилась лишь отчасти. Этот результат мог оказаться иным, если бы не пропуски в данных.

3. Во вкусах пользователей Москвы и Петербурга больше общего чем различий. Вопреки ожиданиям, предпочтения жанров в Петербурге напоминают московские.

Третья гипотеза не подтвердилась. Если различия в предпочтениях и существуют, на основной массе пользователей они незаметны.